GPT ---> Generatively Pretrained Transformer


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F


In [2]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    
print("Length of the text: ", len(text))
    

Length of the text:  1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))

In [5]:
vocab_size = len(chars)

In [6]:
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
print(vocab_size)

65


In [8]:
stoi = {ch: i for i, ch in enumerate(chars)} #mapping from character to integers
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s] #encoding the text into integers
decode = lambda l: ''.join([itos[i] for i in l]) #decoding the integers back to text

In [9]:
print (encode("Hii Nigga"))
print(decode(encode("Hii Nigga")))

[20, 47, 47, 1, 26, 47, 45, 45, 39]
Hii Nigga


In [10]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [11]:
n = int(0.9 * len(data))
train_data = data [:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

In [14]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target: {target}')

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [15]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [16]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is{context.tolist()} the target: {target}')

when input is[24] the target: 43
when input is[24, 43] the target: 58
when input is[24, 43, 58] the target: 5
when input is[24, 43, 58, 5] the target: 57
when input is[24, 43, 58, 5, 57] the target: 1
when input is[24, 43, 58, 5, 57, 1] the target: 46
when input is[24, 43, 58, 5, 57, 1, 46] the target: 43
when input is[24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is[44] the target: 53
when input is[44, 53] the target: 56
when input is[44, 53, 56] the target: 1
when input is[44, 53, 56, 1] the target: 58
when input is[44, 53, 56, 1, 58] the target: 46
when input is[44, 53, 56, 1, 58, 46] the target: 39
when input is[44, 53, 56, 1, 58, 46, 39] the target: 58
when input is[44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is[52] the target: 58
when input is[52, 58] the target: 1
when input is[52, 58, 1] the target: 58
when input is[52, 58, 1, 58] the target: 46
when input is[52, 58, 1, 58, 46] the target: 39
when input is[52, 58, 1, 58, 46, 39] the target: 58
when input i

In [17]:
print(xb) #input to the transformer
print(yb), print(xb.shape), print(yb.shape)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
torch.Size([4, 8])
torch.Size([4, 8])


(None, None, None)

In [18]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding (vocab_size, vocab_size)
    
    def forward(self,idx, targets = None):
        
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
            
        else:
        
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx)
            
            logits = logits[: ,-1, :]
            
            probs = F.softmax(logits, dim = -1)
            
            idx_next = torch.multinomial(probs, num_samples = 1)
            
            idx = torch.cat((idx, idx_next), dim = 1)
            
        return idx
            
    
m = BigramLanguageModel(vocab_size)
logits , loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))


torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [19]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [20]:
batch_size = 32
for steps in range(1000):
    
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    loss.backward()

    optimizer.step()
    
print(loss.item())

3.0514681339263916


In [21]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long ), max_new_tokens = 500)[0].tolist()))


MurkoedMyasstocondad.e-mNSInsso&bF o&$;BRjlootZEOK:
;DREm mpenPES:HUCARfounqCEyoAR:ZEN:VhmrdajCAbeSpr ue.HQNGJGefpostrer bk.hpENUCoeldalErPtshenoefeseards:
ARW3ENGXURDUCI n ckcexpyV&jonnd,s,notor owarCInsXElrPjesV.butrPOZEMIh a hToANIZK:Y kpr ierPoEEQUENP?WNGl?oYopig;ENoV3q&jom.L; ik,
Thwayore,er fyYWitJUCay ma O?AqV-
MoivL
OREzX3Dor,EL.xy
Th r ss myoM:
 pZrAQll'vPOutezN
BPe3mo'RMqFhePOxYenY utek,-ZEFFLar agmoPensfoXh hUCoPAQwn.QUFayestywNod p bGPopAllllernqDGudDUP.JaravVDofIxx.c whooraDivRS:
;F


In [22]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1 , keepdim = True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

print('a =')
print(a)
print('b =')
print(b)

print('c =')
print(c)


a =
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c =
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [23]:
torch.manual_seed(1337) #toy example
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [24]:
B

4

In [25]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [26]:
xbow = torch.zeros((B,T,C)) #bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev,0)

In [27]:
wei = torch.tril(torch.ones(T,T)) #tril --> lower triangular ones value tensor
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x #(B,T,T) @ (B,T,C) --> (B,T,C)
torch.allclose(xbow, xbow2, atol=1e-6, rtol=1e-4)

True

In [28]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [29]:
tril = torch.tril(torch.ones(T,T)) #using Softmax
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float ('-inf'))
wei = F.softmax(wei, dim = -1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [30]:
tril = torch.tril(torch.ones(T,T)) #using Softmax
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float ('-inf'))
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3, atol=1e-6, rtol=1e-4)

True

In [31]:
#self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x)
q = query(x)

wei = q @ k.transpose(-2,-1)    # (B,T,H=16) @ (B,H=16,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float ('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v
out.shape


torch.Size([4, 8, 16])

In [32]:
wei[1]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
        [0.0660, 0.0892, 0.0413, 0.6316, 0.1649, 0.0069, 0.0000, 0.0000],
        [0.0396, 0.2288, 0.0090, 0.2000, 0.2061, 0.1949, 0.1217, 0.0000],
        [0.3650, 0.0474, 0.0767, 0.0293, 0.3084, 0.0784, 0.0455, 0.0493]],
       grad_fn=<SelectBackward0>)

In [33]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5


In [34]:
k.var()

tensor(1.0449)

In [35]:
q.var()

tensor(1.0700)

In [36]:
wei.var()

tensor(1.0918)

In [37]:
torch.softmax(torch.tensor([0.1,-0.2, 0.3, -0.2, 0.5]), dim = -1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [38]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim= -1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [39]:
m = nn.Softmax(dim=1)
input = torch.randn(1,2)
output = m(input)
output

tensor([[0.8509, 0.1491]])

In [40]:
class LayerNorm1d:
    
    def __init__(self, dim, eps= 1e-5, momentum = 0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    def __call__(self, x):
        xmean = x.mean(1, keepdim = True)
        xvar = x.var(1, keepdim = True)
        
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        return self.out 
    
    def parameters(self):
        return [self.gamma, self.beta]
    
    
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape
    

torch.Size([32, 100])

In [41]:
x

tensor([[ 0.1335, -0.1059, -0.3824,  ..., -1.3422, -0.1971,  0.8795],
        [-0.0353, -0.7439, -0.3371,  ..., -0.6276, -0.4846,  0.4556],
        [ 0.3069, -1.5010,  1.4898,  ..., -0.6819,  0.9993,  0.8382],
        ...,
        [-1.6080, -1.6324, -0.7634,  ..., -0.9847,  0.0039, -0.8610],
        [-0.2273,  0.0066, -0.2763,  ..., -0.8705, -1.2442, -0.7531],
        [ 0.3054, -0.1505, -0.3809,  ..., -1.4962, -0.7711, -1.0681]])

In [42]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [43]:
x[0,:].mean(), x[0,:].std() 

(tensor(-9.5367e-09), tensor(1.0000))

Entire code


In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16 #hyperparameters
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
device = 'cuda' if torch.cuda.is_available() else 'cpu'


torch.manual_seed(1337)

with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    
chars = sorted(list(set(text))) 
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i:ch for i , ch in enumerate(chars)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])

#train and test splits
data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    
    data = train_data if split == 'split' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])   
    x, y = x.to(device), y.to(device)

    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
            
        self.dropout = nn.Dropout(dropout)
            
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) 
        
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    
    def __init__(self, num_heads, head_size):   
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])        
        self.proj = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear (n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
            
        )
        
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)  
    
    def forward(self, idx, targets = None):
        
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) #B,T,C
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) #T,C
        x = tok_emb + pos_emb
        x = self.blocks(x)
        y = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss


    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        
        return idx

model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M') #total parameters

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step{iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
    xb, yb = get_batch('train')
    logits , loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
context = torch.zeros ((1, 1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens= 2000)[0].tolist()))

0.209729 M
step0: train loss 4.6524, val loss 4.6497
step100: train loss 2.5374, val loss 2.5275
step200: train loss 2.4136, val loss 2.4049
step300: train loss 2.3194, val loss 2.3209
step400: train loss 2.2530, val loss 2.2589
step500: train loss 2.1582, val loss 2.1722
step600: train loss 2.1104, val loss 2.1130
step700: train loss 2.0569, val loss 2.0597
step800: train loss 2.0243, val loss 2.0436
step900: train loss 1.9903, val loss 1.9770
step1000: train loss 1.9623, val loss 1.9526
step1100: train loss 1.9234, val loss 1.9281
step1200: train loss 1.9138, val loss 1.8983
step1300: train loss 1.8966, val loss 1.8847
step1400: train loss 1.8533, val loss 1.8520
step1500: train loss 1.8221, val loss 1.8206
step1600: train loss 1.8135, val loss 1.8253
step1700: train loss 1.8067, val loss 1.8054
step1800: train loss 1.7703, val loss 1.7800
step1900: train loss 1.7813, val loss 1.7761
step2000: train loss 1.7489, val loss 1.7637
step2100: train loss 1.7401, val loss 1.7394
step2200: t